In [45]:
import pyautogui as pg
from win32gui import FindWindow, GetWindowRect

In [46]:
# auxiliary functions
def subP(a: pg.Point, b: pg.Point):
    return pg.position(a.x - b.x, a.y - b.y)
def addP(a: pg.Point, b: pg.Point):
    return pg.position(a.x + b.x, a.y + b.y)
def addPs(*ps: pg.Point):
    return pg.position(sum(p.x for p in ps), sum(p.y for p in ps))
def mulP(p: pg.Point, times: int):
    return pg.position(p.x * times, p.y * times)

def regionAt(p: pg.Point, w: int, h: int):
    return (p.x, p.y, w, h)
def regionBetween(a: pg.Point, b: pg.Point):
    return (a.x, a.y, b.x - a.x, b.y - a.y)

# current info
print("current mouse: ", pg.position())
print("screen  size : ", pg.size())
scrSize = pg.size()

current mouse:  Point(x=1546, y=312)
screen  size :  Size(width=2560, height=1440)


In [47]:

# >> configs

# apex window size, currently only support 1280x720
apex_size = 1280, 720 
# auto locate game, or locate game by mouse
auto_locate_game = True

# Sorted by the order of position
Bloodhound = (0, 0) # 🐶子
Gibraltar = (1, 0) # 直布罗陀
Lifeline = (2, 0) # 奶妈
Pathfinder = (3, 0) # 探路者
Wraith = (4, 0) # 恶灵
Bangalore = (5, 0) # 班加罗尔
Caustic = (1, 0) # 屁男
Mirage = (1, 1) # 幻象
Octane = (2, 1) # 动力牛子
Wattson = (3, 1) # 电妹
Crypto = (4, 1) # 密客
Revenant = (5, 1) # 亡灵
Loba = (0, 2) # 罗芭
Rampart = (1, 2) # 兰伯特
Valkyrie = (4, 2) # 瓦🐔
Seer = (5, 2) # 希尔
# Those who dont have heirloom
    # Ash
    # Fuse
    # Horizon
    # ...

# those you want to get
legendArr = [
    Wraith,
    Valkyrie,
    Octane,
    Loba,
    Crypto,
    Pathfinder,
]

# just get the first one (more stable),
# or get all (more efficient)
just_first = False


# >> advanced configs

lib_prefix = "lib/"
out_prefix = "out/"

# offset from handle to the game screen
    # measured in Win11 150% scale
handle_offset = pg.position(14, 48)

In [48]:
# >> Code Cell A
# get start position (top left corner of game screen)
# dont move the window after this

def getGameScreenPos(offset: pg.Point):
    handle = FindWindow(None, "Apex Legends")
    if handle == 0:
        # Cannot find the window
        return None
    rect = GetWindowRect(handle)
    ret = addPs(offset, pg.position(*rect[:2]))
    return ret

start = None
if auto_locate_game:
    # Version 2 - locate with pywin32
    start = getGameScreenPos(handle_offset)
    if start is None:
        print("Error: Cannot find the game window. Please open the game first.")
    else:
        print("Get game screen position:", start)
else:
    # Version 1 - locate by cursor
        # locate Apex by cursor after 3 seconds
    pg.sleep(3)
    # start = pg.position(scrSize.width/2 - apexSize[0]/2, scrSize.height/2 - apexSize[1]/2)
    start = pg.position()
    print("Get game screen position:", start)
    # pg.moveTo(start)

Get game screen position: Point(x=49, y=705)


In [49]:
# verify start position
pg.moveTo(start)

In [55]:
# do measurements 
'''
oriArr = []
for i in range(0, 12):
    pg.sleep(2)
    pos = pg.position()
    oriArr.append(pos)
    print(i, ":", pos, end=" ")
    if i != 0 and i % 4 == 0:
        print()
'''

# dont measure again

# oriArr = [
#     pg.position(781, 666),
#     pg.position(859, 662),
#     pg.position(942, 661),
#     pg.position(1026, 657),
#     pg.position(1102, 658),
#     pg.position(1182, 659),
#     pg.position(753, 743),
#     pg.position(837, 744),
#     pg.position(913, 746),
#     pg.position(994, 745),
#     pg.position(1072, 745),
#     pg.position(1154, 748),
# ]

# relArr = []
# for ori in oriArr:
#     rel = subP(ori, start)
#     relArr.append(rel)
#     print(rel)

# MAYDO - capture them rather than store them

relArr = [
    pg.position(141, 306),
    pg.position(219, 302),
    pg.position(302, 301),
    pg.position(386, 297),
    pg.position(462, 298),
    pg.position(542, 299),
    pg.position(113, 383),
    pg.position(197, 384),
    pg.position(273, 386),
    pg.position(354, 385),
    pg.position(432, 385),
    pg.position(514, 388),
]

In [56]:
# verify legend positions (with Valkyrie for example)
dogRel = relArr[0]
nextLine = subP(relArr[6], relArr[0])
nextItem = pg.position((relArr[5].x - relArr[0].x)/5, 0)
print("next line: ", nextLine)
print("next item: ", nextItem)

def getLegendPos(x: int, y: int):
    return addPs(start, dogRel, mulP(nextLine, y), mulP(nextItem, x))

# E.g.: Valkyrie - getLegendPos(4, 2)
ValkPos = getLegendPos(4, 2)
pg.moveTo(ValkPos)
print("Move to Valk Position: ", ValkPos)

next line:  Point(x=-28, y=77)
next item:  Point(x=80, y=0)
Move to Valk Position:  Point(x=454, y=1165)


In [57]:
# init the button array
butArr = {
    "esc": 0.9,
    "legend": 0.7,
    "menu": 0.7,
    "yes": 0.75,
    "settings": 0.9,
    "continue": 0.9,
}

def initAllNone():
    for but, _ in butArr.items():
        globals()[but] = None

def captureAll():
    flag = True
    for but, cfd in butArr.items():
        if globals()[but] is None:
            flag = False
            pos = pg.locateCenterOnScreen(lib_prefix + but + ".png",
                                          region=regionAt(start, *apex_size),
                                          confidence=cfd)
            if pos is not None:
                globals()[but] = pos
                print("got", but, ":", pos)
                print("Things left:", [ b for b in butArr.keys() if globals()[b] is None ])
    if flag:
        print("all captured")
    return flag

def printAll():
    for but, _ in butArr.items():
        print(but, ":", globals()[but])

initAllNone()
printAll()

esc : None
legend : None
menu : None
yes : None
settings : None
continue : None


In [58]:
# >> Code Cell B
# captureAll until done

# for i in range(30):
while True:
    pg.sleep(1)
    done = captureAll()
    if done:
        break
printAll()
print("\ncapture done.")

got continue : Point(x=688, y=1137)
Things left: ['esc', 'legend', 'menu', 'yes', 'settings']
got esc : Point(x=102, y=1376)
Things left: ['legend', 'menu', 'yes', 'settings']
got legend : Point(x=678, y=732)
Things left: ['menu', 'yes', 'settings']
got settings : Point(x=1276, y=1366)
Things left: ['menu', 'yes']
got menu : Point(x=1124, y=1374)
Things left: ['yes']
got yes : Point(x=611, y=1183)
Things left: []
all captured
esc : Point(x=102, y=1376)
legend : Point(x=678, y=732)
menu : Point(x=1124, y=1374)
yes : Point(x=611, y=1183)
settings : Point(x=1276, y=1366)
continue : Point(x=688, y=1137)

capture done.


In [59]:
# click things
def moveAndClick(pos: pg.Point):
    pg.moveTo(pos)
    pg.sleep(0.01)
    pg.click()

def locateAndClick(stem: str, cfd: float, region = regionAt(start, *apex_size)):
    pos = pg.locateCenterOnScreen(lib_prefix + stem + ".png",
                                  region=region,
                                  confidence=cfd)
    flag = pos is not None
    if flag:
        moveAndClick(pos)
        # print("clicked", stem, ":", pos)
    return flag

def tryLocating(stem: str, cfd: float, 
                region = regionAt(start, *apex_size), 
                max: int = 3, interval: float = 0):
    for _ in range(max):
        if locateAndClick(stem, cfd, region):
            return True
        pg.sleep(interval)
    return False

def keepLocating(but: str, cfd: float):
    while True:
        pg.sleep(0.001)
        if locateAndClick(but, cfd):
            break

def moveAndClickBut(but: str):
    moveAndClick(globals()[but])

def clickTill(but: str, done):
    while True:
        # TODO make it possible to interrupt
        moveAndClickBut(but)
        if done():
            break

def clickArray(*arr: str):
    for but in arr:
        keepLocating(but, butArr[but])

def clickLegend(x: int, y: int):
    moveAndClick(getLegendPos(x, y))

def typeArray(*arr: str):
    for but in arr:
        pg.press(but)

# TODO make region smaller (for "continue")

# def routine():
#     clickArray("continue", "legend")
#     clickLegend(4, 2)
#     pg.sleep(0.5) # mannul judge for now
#     clickArray("esc", "esc", "settings", "menu", "yes")

# def routine():
#     clickArray("continue", "legend")
#     pg.sleep(0.05)
#     clickLegend(4, 2)
#     pg.sleep(0.7) # mannul judge for now
#     typeArray("esc", "esc", "esc")
#     pg.sleep(0.01)
#     moveAndClickBut("menu")
#     pg.sleep(0.01)
#     moveAndClickBut("yes")

showPa = addPs(start, relArr[0])
showPb = addPs(showPa, mulP(nextLine, 2), mulP(nextItem, 2))
print("showPa", showPa)
print("showPb", showPb)

heirPa = addPs(start, pg.position(1032, 500))
heirPb = addPs(start, pg.position(1121, 569))

def tillLegendsShow():
    pos = pg.locateCenterOnScreen(lib_prefix + "legendShow.png",
                    region=regionBetween(showPa, showPb))
    return pos is not None

stayAtLegend = 0.3

def routineForOne(x, y):
    clickArray("continue")
    clickTill("legend", tillLegendsShow)
    # TODO detect: if no white lines, quit
    clickLegend(x, y)
    pg.sleep(0.08)
    gotcha = tryLocating("heirloom", cfd=0.8, max=3, interval=0, 
                region=regionBetween(heirPa, heirPb))
    # pg.sleep(stayAtLegend) # mannul judge
    if gotcha:
        return True
    typeArray("esc", "esc", "esc")
    pg.sleep(0.001)
    moveAndClickBut("menu")
    pg.sleep(0.001)
    moveAndClickBut("yes")

def routineForAll(legendPosArr: list):
    ret = []
    clickArray("continue")
    clickTill("legend", tillLegendsShow)
    # TODO detect: if no white lines, quit
    for i, pos in enumerate(legendPosArr):
        clickLegend(*pos)
        pg.sleep(0.08)
        gotcha = tryLocating("heirloom", cfd=0.8, max=3, interval=0, 
                    region=regionBetween(heirPa, heirPb))
        # TODO detect: if has heirloom, seen as gotcha
        if not gotcha:
            if ret != []:
                return ret
            typeArray("esc", "esc", "esc")
            pg.sleep(0.001)
            moveAndClickBut("menu")
            pg.sleep(0.001)
            moveAndClickBut("yes")
            return ret
        ret.append(i)
        # judge if really gotcha by the screenshot
        pg.screenshot(out_prefix + "_gotcha " + str(i) + ".png", region=regionAt(start, *apex_size))
        typeArray("esc")
    return ret

showPa Point(x=190, y=1011)
showPb Point(x=294, y=1165)


In [60]:
# get one heirloom and quit when succeed
def GetHeirloom(legendPos: tuple):
    count = 0
    while True:
        print("round", count)
        gotcha = routineForOne(*legendPos)
        if gotcha:
            print("Gotcha!")
            print("Just exit the game to finish. LOL.")
            break
        count += 1

# keep on getting heirloom provided enough glitch time
def GetHeirlooms(legendPosArr: list):
    count = 0
    while True:
        print("round", count)
        gotchaArr = routineForAll(legendPosArr)
        if gotchaArr:
            print("Gotcha! At", gotchaArr)
            print("Just exit the game to finish. LOL.")
            break
        count += 1

In [61]:
# >> Code Cell C
# repeat the routine

try:
    if just_first:
        GetHeirloom(legendArr[0])
    else:
        GetHeirlooms(legendArr)
except KeyboardInterrupt:
    print("Interrupted by user")

round 0
round 1
round 2
round 3
round 4
round 5
round 6
round 7
round 8
Interrupted by user
